In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import numpy as np
import torch

# 1. Load the SQuAD dataset
dataset = load_dataset("squad")

# 2. Load the DistilGPT-2 tokenizer
model_checkpoint = 'distilgpt2'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

/home/khanh/.local/share/virtualenvs/capstone-3qBDIcND/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 3. Preprocess the dataset
max_length = 512
def preprocess_function(examples):
    inputs = [f"Context: {context}\nQuestion: {question}\nAnswer: {answer['text'][0]}"
              for context, question, answer in zip(examples["context"], examples["question"], examples["answers"])]
    return tokenizer(inputs, truncation=True, max_length=max_length, padding=True)

tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

Map: 100%|███████████████████████████████████████████████████████████████| 10570/10570 [00:01<00:00, 7429.96 examples/s]


In [3]:
# 4. Prepare the datasets for training
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(100))

# 5. Load the model
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
model.resize_token_embeddings(len(tokenizer))

# 6. Set up the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 7. Define training arguments
training_args = TrainingArguments(
    output_dir="./TEMP",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16=True,  # Enable if GPU supports it
    push_to_hub=False,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=3,
)
# 8. Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

/home/khanh/.local/share/virtualenvs/capstone-3qBDIcND/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [4]:
import ipywidgets as widgets
from IPython.display import display, HTML

def create_qa_widget():
    context = widgets.Textarea(placeholder="Enter the context", layout={'width': '100%', 'height': '100px'})
    question = widgets.Text(placeholder="Enter your question", layout={'width': '100%'})
    submit = widgets.Button(description="Submit", layout={'width': '100px'})
    output = widgets.Textarea(disabled=True, layout={'width': '100%', 'height': '200px'})

    # Define button click handler
    def on_submit(b):
        output.value = generate_answer(context.value, question.value)

    submit.on_click(on_submit)

    # Create and display the container
    display(HTML("<style>.container { width:100% !important; }</style>"))
    display(widgets.VBox([
        widgets.HTML("<h3>Context:</h3>"), context,
        widgets.HTML("<h3>Question:</h3>"), question,
        submit,
        widgets.HTML("<h3>Answer:</h3>"), output
    ], layout={'width': '100%', 'max_width': '1000px'}))

# Call this function to create and display the widget
create_qa_widget()

ModuleNotFoundError: No module named 'ipywidgets'